In [1]:
import sys
import re
import numpy
import pandas

In [2]:
def get_words_length(text: str) -> list:
    length = []
    
    word_len = 0
    for s in text:
        if s == ' ':
            if word_len != 0:
                length.append(word_len)
            word_len = 0
            continue
        
        word_len = word_len + 1
    
    length.append(word_len)
    
    return length

km_reduce = lambda text: re.sub(' ', '', text)

def recover_spaces(text: str, word_length: list) -> str:
    i, recovered = 0, ''
    
    for s in text:
        if i == word_length[0]:
            recovered = recovered + ' '
            i = 0
            del word_length[0]
            
            if len(word_length) == 0:
                break
    
        recovered = recovered + s
        i = i + 1
    
    return recovered

In [3]:
def transfer_to_codes(word: str, symbols: list) -> list:
    codes = []

    for letter in word:
        for i in range(len(symbols)):
            if letter == symbols[i]:
                codes.append(i) 

    return codes

In [4]:
transfer_to_symbols = lambda word, symbols: ''.join([symbols[x] for x in word])

In [5]:
def visener_encrypt(word: str, key: str, symbols: list) -> str:
    word, key = tuple([transfer_to_codes(x, symbols) for x in [word, key]])
    k, n = tuple(map(len, [key, symbols]))

    return transfer_to_symbols([(word[i] + key[i % k]) % n for i in range(len(word))], symbols)

In [6]:
def visener_decrypt(word: str, key: str, symbols: list) -> str:
    word, key = transfer_to_codes(word, symbols), transfer_to_codes(key, symbols)
    k, n = len(key), len(symbols)

    return transfer_to_symbols([(word[i] - key[i % k]) % n for i in range(len(word))], symbols)

In [7]:
def shift(word: str, s: int, symbols: list) -> str:
    word = transfer_to_codes(word, symbols)
    n = len(symbols)
    
    return transfer_to_symbols([(x + s) % n  for x in word], symbols)

In [8]:
def km_split(text: str, k: int) -> list:
    spl = []

    for i in range(k):
        spl.append(text[i::k])

    return spl

In [9]:
def km_count_freqs(text: str, symbols: list) -> list:
    freqs = [0 for x in symbols]

    for i in range(len(symbols)):
        for s in text:
            if symbols[i] == s:
                freqs[i] = freqs[i] + 1

    return freqs

In [10]:
def calc_index_of_coincidence(text: str, symbols: list):
    freqs = km_count_freqs(text, symbols)
    n = len(text)
    index = numpy.sum([x * (x - 1) for x in freqs]) / (n * (n - 1))
    
    return index

In [11]:
def apply_partitions(text: str, n: int, symbols: list):
    table = [[], []]
    for k in range(1, n + 1):
        table[0].append(k)
        index =numpy.average([calc_index_of_coincidence(x, symbols) for x in km_split(text, k)])
        table[1].append(index)
    
    return pandas.DataFrame(table[1], index=table[0], columns=['index_of_coincidence'])

In [12]:
def calc_mutual_index_of_coincidence(text1: str, text2: str, symbols: list):
    freqs1 = km_count_freqs(text1, symbols)
    freqs2 = km_count_freqs(text2, symbols)
    n, m = len(text1), len(text2)
    mi = numpy.sum([x * y for x, y in zip(freqs1, freqs2)]) / (n*m)
    
    return mi

In [13]:
def get_mutual_shift(text1: str, text2: str, symbols: list):
    table = [[], []]
    m = len(symbols)
    
    for k in range(1, m + 1):
        table[0].append(k)
        table[1].append(calc_mutual_index_of_coincidence(shift(text1, k, symbols), text2, symbols))
    
    return pandas.DataFrame(table[1], index=table[0], columns=['mutual_index_of_coincidence'])

In [ ]:
if __name__ == "__main__":
    symbols = [chr(x) for x in range(32, 127)]
    m = len(symbols)
    
    f = open('src.txt')
    text, key = f.read(), 'conception'
    enc_text = visener_encrypt(text, key, symbols) 

    frame= apply_partitions(enc_text, 30, symbols)
    
    f.close()

In [ ]:
print(get_spaces_positions(text[:500]))

In [ ]:
frame.nlargest(10, 'index_of_coincidence')

In [ ]:
d = 12
lines = km_split(enc_text, d)

In [ ]:
shifts = [0]
for i in range(1, d):
    shifts.append(get_mutual_shift(lines[0], lines[i], symbols).nlargest(1, 'mutual_index_of_coincidence').index[0])
print(shifts)

In [ ]:
# k1 k2 ... kn
# s1 = 0
# si - относительные сдвиги
# k1 - k2 = s2
# k1 - k3 = s3
# ...
# k1 - kn = sn

In [ ]:
transfer_to_codes(key, symbols)

In [ ]:
for i in range(m):
    key = []
    for x in shifts:
        key.append((x + i) % m)
    
    print(str(i) + ': ' + transfer_to_symbols(key, symbols))